# Machine Learning A-Z: Section 5 Multiple Linear Regression

Multiple Linear Regression is similar to Simple Linear Regression from Section 4. The major difference is that instead of trying to find a linear relationship between a single independent variable and a single dependent variable, we are looking for a relationship between a linear combination of multiple independent variables and a single dependent variable. Like trying to fit a line to a cluster of datapoints in 3D+ space instead of 2D space.

## Step 1 Import and Prepare the data.

We'll use the template we created in Section 2 to import and preprocess the data.

In [1]:
import numpy as np # Libraries for fast linear algebra and array manipulation
import pandas as pd # Import and manage datasets
from plotly import __version__ as py__version__
import plotly.express as px # Libraries for ploting data
import plotly.graph_objects as go # Libraries for ploting data
from sklearn import __version__ as skl__version__
from sklearn.preprocessing import OneHotEncoder # Libraries to do encoding of categorical variables
from sklearn.compose import ColumnTransformer # Library to transform only certain columns/features at a time
from sklearn.model_selection import train_test_split # Library to split data into training and test sets.
from sklearn.linear_model import LinearRegression # Library for creating Linear Regression Models
from statsmodels import __version__ as statsmodels__version__
import statsmodels.api as sm

Library versions used in this code:

In [2]:
print('Numpy: ' + np.__version__)
print('Pandas: ' + pd.__version__)
print('Plotly: ' + py__version__)
print('Scikit-learn: ' + skl__version__)
print('Stats Models: ' + statsmodels__version__)

Numpy: 1.16.4
Pandas: 0.25.1
Plotly: 4.0.0
Scikit-learn: 0.21.2
Stats Models: 0.10.1


In [3]:
def LoadData():
    dataset = pd.read_csv('50_Startups.csv')
    return dataset

dataset = LoadData()
print(dataset.head(3))
print()
print(dataset.info())

   R&D Spend  Administration  Marketing Spend       State     Profit
0  165349.20       136897.80        471784.10    New York  192261.83
1  162597.70       151377.59        443898.53  California  191792.06
2  153441.51       101145.55        407934.54     Florida  191050.39

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
R&D Spend          50 non-null float64
Administration     50 non-null float64
Marketing Spend    50 non-null float64
State              50 non-null object
Profit             50 non-null float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB
None


In [4]:
X = dataset.iloc[:,:-1].values # All the columns except the last are features
y = dataset.iloc[:,-1].values # The last column is the dependent variable

#Do the One-Hot encoding on our categorical data.
columntransformer = ColumnTransformer(
    [('Country_Category', OneHotEncoder(), [3])],
    remainder = 'passthrough')

X = np.array(columntransformer.fit_transform(X))

#Remove one of the new dummy variables to avoid the dummy vatiable trap.
X = X [:,1:]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

Notes about the preprocessing:
* We skipped the following sections of preprocessing:
  * Missing Data - The dataset is complete with no missing data
  * Feature Scaling - The linear regression libraries used here do not require prescaled data
* After One-Hot encoding the new columns are as follows:
  1. California
  2. Florida
  3. New York
* To avoid the Dummy Variable Trap (i.e. Multicollinearity) we removed the first column (California) Hence we are left with the following independent variables:
  1. Florida
  2. New York
  3. R&D Spend
  4. Administration
  5. Marketing Spend

## Step 2: Fit a Simple Linear Regression Model

In [6]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Step 3: Evalulate the Model

In [7]:
y_pred_test = regressor.predict(X_test)

for p, a in zip(y_pred_test, y_test):
    print("{0:.0f}\t{1:.0f}\t{2:.0f}\t{3:.1f}%".format(p,a,p-a, a/p*100))

126363	134307	-7944	106.3%
84608	81006	3603	95.7%
99677	99938	-260	100.3%
46357	64926	-18569	140.1%
128750	125370	3380	97.4%
50912	35673	15239	70.1%
109741	105734	4008	96.3%
100643	107404	-6761	106.7%
97599	97428	171	99.8%
113097	122777	-9679	108.6%


It looks like in general our model is fairly acurate, but there are a few instances where we were way off. In one case a start-up out performed the prediction by 40% and in another it underperformed by 30%. In general though, our model was within about 5% of the actual profit.

However we should look at the statistical significance of wach independent variable to see if we can eliminate any.

## Step 4: Independent Variable P-Values & Backward Elimination

We are going to use StatsModel to calculate the _p-value_ for each of our independent variables. However in order to do that, stat's model needs to know the value of the constant term the regressor fitted. To find that we can give the regressor another dummy variable that is always one to represent the constant value and use a linear regressor that doesn't find the intercept.

In [8]:
X = np.append(values = X.astype(float), arr = np.ones((len(X),1)).astype(int), axis = 1)

In [11]:
X_opt = X[:,[0,1,2,3,4,5]]

regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Thu, 05 Sep 2019   Prob (F-statistic):           1.34e-27
Time:                        22:59:54   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
x1           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
x2           -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
x3             0.8060      0.046     17.369      0.000       0.712       0.900
x4            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x5             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Looking at the _p-values_ we see that x2 (our dummy variable for New York) has the largest _p-value_ with 0.990 which is above our significance level of 0.05. Hence due to the backward elimination process we are going to remove x2 and refit the model

In [13]:
X_opt = X[:,[0,1,3,4,5]]

regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Thu, 05 Sep 2019   Prob (F-statistic):           8.49e-29
Time:                        23:08:48   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.011e+04   6647.870      7.537      0.000    3.67e+04    6.35e+04
x1           220.1585   2900.536      0.076      0.940   -5621.821    6062.138
x2             0.8060      0.046     17.606      0.000       0.714       0.898
x3            -0.0270      0.052     -0.523      0.604      -0.131       0.077
x4             0.0270      0.017      1.592      0.118      -0.007       0.061
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Continuing the Backwards Elimination we see that x1 (dummy variable for Florida) has the highest _p-value_ which is still above our 0.05 significance level. So we remove it and continue.

In [14]:
X_opt = X[:,[0,3,4,5]]

regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Thu, 05 Sep 2019   Prob (F-statistic):           4.53e-30
Time:                        23:11:21   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.012e+04   6572.353      7.626      0.000    3.69e+04    6.34e+04
x1             0.8057      0.045     17.846      0.000       0.715       0.897
x2            -0.0268      0.051     -0.526      0.602      -0.130       0.076
x3             0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now we see that x2 (Administration Spend) has the highest _p-value_ of 0.602 so we remove it and continue again.

In [15]:
X_opt = X[:,[0,3,5]]

regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Thu, 05 Sep 2019   Prob (F-statistic):           2.16e-31
Time:                        23:13:45   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
x1             0.7966      0.041     19.266      0.000       0.713       0.880
x2             0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Here we see that x2 (Marketing Spend) has the highest _p-value_ of 0.06 which is still greater than our significance level of 0.05 so we remove it and continue.

In [16]:
X_opt = X[:,[0,3]]

regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Thu, 05 Sep 2019   Prob (F-statistic):           3.50e-32
Time:                        23:15:58   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.000    4.39e+04    5.41e+04
x1             0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

And finally we are left with a model in which all the remaining independent variables have a _p-value_ less than 0.05 and our model has been reduced from a multiple linear regression to a simple linear regression showing that R&D spend has the only statistically significant affect (at the p < 0.05 level) on profit.

## Linear Regression Assumptions:

1. **Linearity**: The relationship between independent & dependent variables is able to be represented by a straight line.
2. **Homoscedasticity** : All points have approximately the same variance. (i.e. the data looks like a tube not a cone)
3. **Multivariate Normality**: The independent variables all have normal distributions
4. **Independence of Errors**: The residuals shouldn't show any correlation with time, space, or any other variables
5. **Lack of Multicollinearity**: Also called linear independence. None of your independent variable should be predictable from some combination of the remaining independent variables.